In [5]:
import sqlite3
from rich.console import Console
from rich.table import Table
from rich.style import Style
from rich.text import Text

def query_payment_data(cursor, params):
    """
    查询支付数据
    
    Args:
        cursor: 数据库游标
        params: 查询参数字典
    
    Returns:
        tuple: (success, result)
        - success: bool, 查询是否成功
        - result: 成功时为数据列表，失败时为错误信息
    """
    try:
        # 检查表是否存在
        table_name = f"pay_{params['monthyear']}"
        cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' AND name=?
        """, (table_name,))
        
        if not cursor.fetchone():
            return False, f"表格 {table_name} 不存在"
        
        # 构建查询条件
        conditions = []
        query_params = []
        
        # 标签筛选
        # if params['tag'] and params['tag'] != "N":
        if params['tag'] :
            conditions.append("pay_tag = ?")
            query_params.append(params['tag'])
        
        # 关键词搜索
        if params['key']:
            conditions.append("(pay_source LIKE ? OR pay_note LIKE ?)")
            query_params.extend([f"%{params['key']}%", f"%{params['key']}%"])
        
        # 金额范围筛选
        if params['lt']:
            try:
                lt_value = float(params['lt'])
                conditions.append("pay_money < ?")
                query_params.append(lt_value)
            except ValueError:
                return False, "金额上限格式错误"
        
        if params['gt']:
            try:
                gt_value = float(params['gt'])
                conditions.append("pay_money > ?")
                query_params.append(gt_value)
            except ValueError:
                return False, "金额下限格式错误"
        
        # 构建完整的SQL查询
        query = f"SELECT * FROM {table_name}"
        if conditions:
            query += " WHERE " + " AND ".join(conditions)
        query += " ORDER BY pay_money DESC"
        
        # 执行查询
        cursor.execute(query, query_params)
        
        # 获取列名
        columns = [description[0] for description in cursor.description]
        
        # 将结果转换为字典列表
        results = []
        for row in cursor.fetchall():
            results.append(dict(zip(columns, row)))
        
        return True, results
        
    except sqlite3.Error as e:
        return False, f"数据库错误: {str(e)}"

def display_query_results(results, params):
    """
    使用rich库显示查询结果
    """
    console = Console()
    
    if not results:
        console.print("[red]没有找到数据[/red]")
        return
    
    # 创建表格
    table = Table(
        title=f"支付记录 ({params['monthyear']})",
        show_header=True,
        header_style="bold magenta",
        border_style="blue",
        title_style="bold cyan"
    )
    
    # 添加列
    # table.add_column("ID", style="dim")
    table.add_column("日期", style="cyan")
    table.add_column("金额", justify="right", style="green")
    table.add_column("来源", style="yellow")
    table.add_column("标签", style="red")
    table.add_column("备注")
    
    # 添加数据行
    for row in results:
        money = f"¥{float(row['pay_money']):.2f}"
        
        # 根据金额设置不同样式
        if float(row['pay_money']) > 100:
            money_style = "[bold red]" + money + "[/bold red]"
        else:
            money_style = "[green]" + money + "[/green]"
            
        table.add_row(
            # str(row['id']),
            row['pay_time'],
            money_style,
            row['pay_source'],
            row['pay_tag'],
            row.get('pay_note', '')
        )
    
    # 显示查询条件
    console.print("\n[bold yellow]查询条件:[/bold yellow]")
    conditions = []
    if params['tag'] and params['tag'] != "N":
        conditions.append(f"标签: {params['tag']}")
    if params['key']:
        conditions.append(f"关键词: {params['key']}")
    if params['lt']:
        conditions.append(f"金额 < {params['lt']}")
    if params['gt']:
        conditions.append(f"金额 > {params['gt']}")
    
    if conditions:
        console.print(" | ".join(conditions))
    
    # 打印表格
    console.print("\n")
    console.print(table)
    
    # 打印统计信息
    total_amount = sum(float(row['pay_money']) for row in results)
    console.print(f"\n[bold green]总金额: ¥{total_amount:.2f}[/bold green]")
    console.print(f"[dim]共 {len(results)} 条记录[/dim]")

# 使用示例
def main():
    # 测试参数
    params = {
        "monthyear": "202501",
        "tag": "",
        "key": "",
        "lt": "",
        "gt": ""
    }
    
    # 创建数据库连接
    conn = sqlite3.connect('moneycount.db')
    cursor = conn.cursor()
    
    # 执行查询
    success, result = query_payment_data(cursor, params)
    
    if success:
        display_query_results(result, params)
    else:
        console = Console()
        console.print(f"[red]错误: {result}[/red]")
    
    # 关闭连接
    conn.close()

if __name__ == "__main__":
    main()

查询条件:

                                                 支付记录 (202501)                                                 
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 日期                 ┃     金额 ┃ 来源                            ┃ 标签       ┃ 备注                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2025-01-05 15:44:24  │ ¥1500.00 │ 🍀                              │ N          │ 转账备注:2月份房租             │
│ 2025-01-21 14:36:00  │  ¥405.34 │ 广东省税务局                    │ N          │ 基本医疗保险费|44401625010308… │
│ 2025-01-25 15:13:14  │  ¥346.77 │ 铁路12306                       │ traffic_ud │ 火车票                         │
│ 2025-01-01 21:39:53  │  ¥325.10 │ Steam                           │ N          │ Steam Purchase                 │
│ 2025-01-19 10:17:29  │  ¥303.90 │ 铁路12306                       │ traffic_ud │ 火车票                         │
│ 2025-01-08 19:51:11  │  ¥285.00 │ B 赖雨馨                        │ N          │ 转账备注:微信转账              │
│ 2025-01-04 17:38:27  │  ¥100.00 │ 手机充值                        │ N          │ 100元手机话费-13142202019      │
│ 2025-01-14 14:33:22  │   ¥80.00 │ Alan                            │ N          │ 收钱码收款                     │
│ 2025-01-06 18:36:17  │   ¥62.00 │ 喜太熊（萝岗万达店）            │ N          │ 经营码交易                     │
│ 2025-01-04 17:38:39  │   ¥50.00 │ 手机充值                        │ N          │ 50元手机话费-15626203788       │
│ 2025-01-15 00:09:05  │   ¥45.00 │ 铁臂小乖乖的店                  │ N          │ 收钱码收款                     │
│ 2025-01-06 17:20:45  │   ¥39.90 │ 泽合**店                        │ N          │ 马来西亚原装进口泽合怡保白咖 … │
│ 2025-01-06 16:15:23  │   ¥38.00 │ Apple                           │ N          │ 查询购买记录：reportaproblem.… │
│                      │          │                                 │            │ 如需帮助请访问：support.apple… │
│ 2025-01-24 14:45:50  │   ¥34.89 │ 钱大妈                          │ eat_lunch  │ 蒙牛每日鲜酪牛油果味发酪乳100… │
│ 2025-01-07 13:33:18  │   ¥30.80 │ 美团                            │ eat_lunch  │ 香源烧鸡(乳山生蚝)-美团App-25… │
│ 2025-01-23 14:14:48  │   ¥28.90 │ 美团                            │ eat_lunch  │ 肯德基-美团App-25012311100400… │
│ 2025-01-13 14:20:17  │   ¥28.25 │ 钱大妈                          │ eat_lunch  │ 青瓜等3件商品                  │
│ 2025-01-15 16:05:39  │   ¥28.00 │ 萨莉亚意大利餐厅(279高德置地春… │ N          │ GZ279高德置地春广场店          │
│ 2025-01-21 15:18:55  │   ¥27.99 │ 美团                            │ eat_lunch  │ 肯德基-美团App-25012111100400… │
│ 2025-01-20 16:04:13  │   ¥27.90 │ 美团                            │ eat_lunch  │ 肯德基-美团App-25012011100400… │
│ 2025-01-08 15:03:55  │   ¥26.35 │ 钱大妈                          │ eat_lunch  │ 青虾仁等2件商品                │
│ 2025-01-18 15:18:23  │   ¥24.54 │ 钱大妈                          │ eat_lunch  │ 青虾仁等2件商品                │
│ 2025-01-09 16:01:46  │   ¥22.98 │ 淘宝平台商户                    │ N          │ /                              │
│ 2025-01-15 14:58:43  │   ¥22.00 │ 俏碗里·馄饨面点（猎德店）       │ N          │ 美团/大众点评点餐订单-0115471… │
│ 2025-01-17 10:51:07  │   ¥20.00 │ Apple                           │ N          │ 查询购买记录：reportaproblem.… │
│                      │          │                                 │            │ 如需帮助请访问：support.apple… │
│ 2025-01-17 10:26:55  │   ¥19.00 │ Apple                           │ N          │ 查询购买记录：reportaproblem.… │
│                      │          │                                 │            │ 如需帮助请访问：support.apple… │
│ 2025-01-16 20:50:32  │   ¥17.42 │ 广州帕比亚餐饮管理有限公司      │ N          │ 消费：帕比亚・天河区棠东店   … │
│ 2025-01-19 14:30:19  │   ¥16.26 │ 钱大妈                          │ eat_lunch  │ 嫩滑鸡柳丝200g等2件商品        │
│ 2025-01-22 09:58:34  │   ¥15.77 │ 钱大妈                          │ eat_lunch  │ 嫩滑鸡柳丝200g等2件商品        │
│ 2025-01-09 14:40:56  │   ¥15.24 │ 钱大妈                          │ eat_lunch  │ 嫩滑鸡柳丝200g等2件商品        │
│ 2025-01-11 14:43:36  │   ¥15.18 │ 钱大妈                          │

总金额: ¥4140.86

共 43 条记录